In [1]:
import dask.dataframe as dd
import dask
import dask.array as da
import pandas as pd

from dask.distributed import Client
from datetime import datetime
from pathlib import Path
import numpy as np

client = Client(n_workers=4)

# CADES configuration
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(project='birthright', queue='high_mem_cd', cores=30, memory='150 GB', job_extra=["-N 1"], interface="ib0")
# client = Client(cluster)

In [2]:
# # Generate the square footage data table from the raw npy files.
# def make_sqft_dict():
#     dict_sqft = {
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_1.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_2.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_3.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_4.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_5.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_6.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_7.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_8.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_9.npy', allow_pickle=True).flatten()))[0],
#         **dict(enumerate(np.load('data/reference/sq_ft/dict_10.npy', allow_pickle=True).flatten()))[0]
#     }
#     return(dict_sqft)
# dict_sqft = make_sqft_dict()
# # Do this nonsense with the indexes to get them persisted to parquet
# df = pd.DataFrame.from_dict(dict_sqft, orient='index', columns=['sqft36']).reset_index().set_index('index')
# # df.head()
# df.to_parquet("data/reference/sq_ft/sg_sqft.parquet")

In [2]:
def to_date(d):
    if ":" == d[-3:-2]:
        d = d[:-3]+d[-2:]
    return d

In [3]:
patterns = dd.read_csv("data/safegraph/weekly-patterns/2020-04*.csv", dtype={'naics_code': 'float64'}).set_index('safegraph_place_id')
patterns['date'] = patterns['date_range_start'].apply(lambda x: datetime.strptime(to_date(x), '%Y-%m-%dT%H:%M:%S%z').date(), meta=('date_range_start', 'object'))

In [4]:
sq_ft = dd.read_parquet("data/reference/sq_ft/sg_sqft.parquet")

In [5]:
# do an inner join to focus on the POIs that we actually have data for.
df = dd.merge(patterns, sq_ft)
df = df.reset_index()
df = df.rename(columns={'index': 'safegraph_place_id'})

In [6]:
def strings_to_int(lists):
    ints = []
    for x in lists:
        try:
            ints.append(int(x))
        except:
            ints.append(0)
    return np.array(ints).astype(np.int32)

def get_cbgs(df):   
    df = df[['safegraph_place_id', 'date','visitor_home_cbgs', 'visits_by_day']]
    df = df.visitor_home_cbgs\
        .str.translate(str\
        .maketrans({'{':'', '}':'','"':''}))\
        .str.split(',')\
        .apply(pd.Series, 1)\
        .stack() \
        .reset_index(level=1, drop=True) \
        .to_frame('cbgs') \
        .merge(df, left_index=True, right_index=True)\
        .drop(['visitor_home_cbgs'], axis=1) \
        .rename(columns={0:'cbgs',1:'party'}) \
        .dropna()
    
    df = df[df.cbgs != ""]
    df['party'] = df.cbgs.str.split(':').apply(lambda x: x[-1]).astype(np.int32)
    df['cbgs'] = df.cbgs.str.split(':').apply(lambda x: x[0]).astype('str')

    return(df)

df_blocks = df.map_partitions(get_cbgs, meta={'cbgs': 'str', 'safegraph_place_id': 'str', 'date': 'object', 'visits_by_day': 'object', 'party': 'int32'})

In [7]:
df_firms = df[['safegraph_place_id','location_name','brands','naics_code']].drop_duplicates()

In [8]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def get_contacts(df):
    df = df[['safegraph_place_id', 'visits_by_each_hour','median_dwell','date','sqft36']].copy()
    # Generate list of visits by hour
    df['close_contacts'] = None
    # Turn string into a list of strings
    df.close_contacts = df.visits_by_each_hour.str.translate(str.maketrans({'[':'', ']':'','"':''})).str.split(',')
    # Turn list of strings into a list of ints with each int indicating visitors during each hour of the week
    df.close_contacts = df.apply(lambda row: [int(i) for i in row.close_contacts], axis=1)
    # Turn the list of ints into a chunks of 24 hours for each day and calculate the average interactions per 36 square feet
    df.close_contacts = df.apply(lambda row: [np.divide(\
                                              pd.DataFrame(c).rolling(int(max(row.median_dwell/60, 1)), min_periods=1).sum()[0].tolist()\
                                              , row.sqft36)\
                                              .mean()
                                              for c in chunks(row.close_contacts, 24)], axis=1)
    df = df.drop(['visits_by_each_hour','median_dwell','sqft36'], axis = 1)
    df = df.close_contacts.apply(pd.Series)\
         .merge(df, left_index=True, right_index=True)\
         .drop(['close_contacts'], axis=1)\
         .melt(id_vars=['safegraph_place_id','date'],value_name='visits')
    df['d2'] = (df.date + pd.to_timedelta(pd.np.ceil(df.variable.astype(float)), unit="D"))
    df = df.drop(['variable'], axis = 1)
    print(df)
    return (df)

df_contacts = df.map_partitions(get_contacts, meta={'safegraph_place_id': 'str', 'date': 'object', 'visits': 'int32', 'd2': 'object'})

In [ ]:
df = df_contacts.merge(df_blocks, on=['safegraph_place_id','date'])
df = df.merge(df_firms, left_on='safegraph_place_id', right_on='safegraph_place_id') 
df.head()